# Generate Reports for E*Trade transactions

E*Trade allows you to download your transaction data.  Here are some
examples of using some data to generate reports related to how much
was paid infees (for their "managed" account) and yearly dividends.

In [ ]:
import pandas as pd
import glob
import os
import sys
!{sys.executable} -m pip install yfinance

## Read in the Data

Before running, you'll have to download your E*Trade date and place it in a
subdirectory.

Edit each csv file and remove the first two lines so that the
header is the first line.

In [ ]:
dr = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "etrade/*.csv"))))

In [ ]:
dr.TransactionDate = pd.to_datetime(dr.TransactionDate)

In [ ]:
for year in range(2017, 2021, 1):
    
    # Filter on the year separately to enhance readability.
    #
    year_index = ( (dr.TransactionDate >= f'{year}/01/01') &
                   (dr.TransactionDate <= f'{year}/12/31') )
    
    fee_frame = dr[ year_index & (dr.TransactionType == 'Fee')][['TransactionDate', 'Amount']]
    print(fee_frame.sort_values('TransactionDate'))
    fee_total = fee_frame.Amount.sum()
    print(f"{'*' * 40} Total fees for {year}: ${fee_total:,.2f}\n")
    
    div_frame = dr[ year_index & (dr.TransactionType == 'Dividend')][['TransactionDate', 'SecurityType', 'Symbol', 'Amount']]
    print(div_frame.sort_values('TransactionDate'))
    div_total = div_frame.Amount.sum()
    print(f"{'*' * 40} Total dividends for {year}: ${div_total:,.2f}\n")
    

In [ ]:
dr [ (dr.TransactionType != 'Dividend') &
     (dr.TransactionType != 'Bought') &
     (dr.TransactionType != 'Sold') &
     (dr.TransactionType != 'Fee') ]

In [ ]:
dr [dr.TransactionType == 'Other']['Description']

## Get the Security Symbols

In [ ]:
# Get the list of security symbols, put them into a list and then
# get the unique ones.
#
symbol_set = set(
     dr [ (dr.TransactionType == 'Dividend') |
          (dr.TransactionType == 'Bought') |
          (dr.TransactionType == 'Sold')]['Symbol'].tolist())

symbol_set

In [ ]:
# For each symbol, build a dict of the actual security name.
#
import yfinance as yf

symbol_dict = {}
def print_sec(aSymbol):
    try:
        val = yf.Ticker(aSymbol).info['longName']
    except KeyError as e:
        print(f"Error on {aSymbol}", e)
        val = "Unknown"
    print(f"{aSymbol:8s} {val}")
    symbol_dict[aSymbol] = val
    
for s in sorted(symbol_set):
    print(s)
    print_sec(s)

In [ ]:
for k,v in symbol_dict.items():
    print(k,v)